In [ ]:
# --- PASO 0: Importar Librerías ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from catboost import CatBoostClassifier

# --- Librerías para Preprocesamiento Avanzado ---
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer # Para Transformación (normal) y Estandarización
from sklearn.pipeline import Pipeline # Usaremos pipelines para simplificar
from imblearn.over_sampling import SMOTE # Para balanceo avanzado

import warnings

# Configuración
warnings.simplefilter(action='ignore', category=FutureWarning)
sns.set(style="darkgrid")
print("Librerías importadas.")

# ---------------------------------------------------------------------------
# --- PASO 1: Entender las columnas del archivo CSV ---
# ---------------------------------------------------------------------------
print("\n--- 1. Cargando y Explorando los Datos ---")
try:
    df = pd.read_csv('data/water_potability.csv')
    print("Archivo 'data/water_potability.csv' cargado exitosamente.")
except FileNotFoundError:
    print("ERROR: 'data/water_potability.csv' no encontrado.")
    print("Asegúrate de que el archivo esté en la ruta correcta antes de continuar.")
    raise SystemExit("Deteniendo ejecución: archivo no encontrado.")

print("\n--- Información General (Tipos de Dato y Conteo) ---")
df.info()

print("\n--- Conteo de Valores Nulos (Missing) ---")
print(df.isna().sum())

print("\n--- Desbalance de Clases (Objetivo 'Potability') ---")
print(df['Potability'].value_counts(normalize=True))

# ---------------------------------------------------------------------------
# --- PASO 2: Selección de Características (Método Embebido) ---
# ---------------------------------------------------------------------------
print("\n--- 2. Implementando Selección de Características (Método Embebido) ---")
# Usaremos CatBoost (un método embebido) para obtener la importancia de características.
# Para hacerlo correctamente, primero debemos preprocesar los datos.

# 2a. Preparar datos para el modelo selector
print("Preparando datos para el modelo selector...")
X = df.drop('Potability', axis=1)
y = df['Potability']
feature_names = X.columns # Guardar nombres para después

# 2b. Dividir en Train y Test (ANTES de cualquier preprocesamiento)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Datos divididos: {len(X_train)} para entrenamiento, {len(X_test)} para prueba.")

# 2c. Pipeline de Preprocesamiento Avanzado
# 1. KNNImputer (Imputación avanzada)
# 2. PowerTransformer (Transformación a distribución normal y Estandarización)
print("Creando pipeline de preprocesamiento (Impute -> Transform/Standardize)...")
preprocessing_pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)),
    ('transformer', PowerTransformer(method='yeo-johnson', standardize=True)) # Estandariza (Normaliza)
])

# Ajustar (fit) el pipeline SÓLO con datos de train (evita data leakage)
print("Ajustando pipeline de preprocesamiento SÓLO en datos de train...")
X_train_processed = preprocessing_pipeline.fit_transform(X_train)
# Transformar el set de prueba
X_test_processed = preprocessing_pipeline.transform(X_test)

# Convertir de nuevo a DataFrames
X_train_processed = pd.DataFrame(X_train_processed, columns=feature_names)
X_test_processed = pd.DataFrame(X_test_processed, columns=feature_names)
print("Datos de entrenamiento y prueba preprocesados.")

# 2d. Balanceo de Clases Avanzado (SMOTE)
# Se aplica SÓLO al set de entrenamiento
print("Aplicando balanceo de clases (SMOTE) al set de entrenamiento...")
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_processed, y_train)
print(f"Tamaño de Train antes de SMOTE: {X_train_processed.shape[0]}")
print(f"Tamaño de Train después de SMOTE: {X_train_balanced.shape[0]}")

# 2e. Entrenar modelo "Selector" para obtener importancias
# (Este modelo se entrena con los datos preprocesados, balanceados y TODAS las características)
print("\nEntrenando modelo selector para obtener importancias...")
selector_model = CatBoostClassifier(
    iterations=500, learning_rate=0.05, eval_metric='Accuracy',
    random_seed=42, verbose=0,
    allow_writing_files=False # Evitar archivos de log
)

# Usamos los datos balanceados para el entrenamiento del selector
selector_model.fit(
    X_train_balanced, y_train_balanced,
    eval_set=(X_test_processed, y_test),
    early_stopping_rounds=50
)

# 2f. Obtener y mostrar resultados de importancia
print("\n--- Resultados de Importancia de Características ---")
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': selector_model.get_feature_importance()
})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print(importance_df.to_string())

# 2g. Decisión: Nos quedamos con las 5 características principales
# (Basado en la imagen 'image_fb9c0a.png', las 5 primeras tienen > 10% de importancia)
features_to_keep = importance_df.head(5)['Feature'].tolist()
print(f"\nACCIÓN: Se seleccionan las 5 mejores características: {features_to_keep}")

# ---------------------------------------------------------------------------
# --- PASO 3: Optimización y Entrenamiento del Modelo FINAL ---
# ---------------------------------------------------------------------------
print("\n--- 3. Optimizando y Entrenando Modelo FINAL (sobre 5 características) ---")

# 3a. Reducir los DataFrames a las características seleccionadas
X_train_final = X_train_balanced[features_to_keep]
X_test_final = X_test_processed[features_to_keep] # El Test set NO se balancea

# 3b. Definir los parámetros a probar (GridSearchCV)
param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.03, 0.05, 0.1],
    'iterations': [500, 1000]
}

# 3c. Inicializar el modelo base para el GridSearch
# No usamos 'scale_pos_weight' porque ya balanceamos con SMOTE
grid_search_model = CatBoostClassifier(
    random_seed=42,
    eval_metric='F1', # Optimizamos para F1, mejor métrica en desbalance
    verbose=0,
    allow_writing_files=False
)

# 3d. Configurar y ejecutar GridSearchCV
grid_search = GridSearchCV(
    estimator=grid_search_model,
    param_grid=param_grid,
    cv=3, # 3-Fold Cross-Validation
    scoring='f1_weighted', # Usamos F1 ponderado para la búsqueda
    n_jobs=-1 # Usar todos los núcleos de CPU
)

print("Iniciando GridSearchCV (esto puede tardar)...")
# Entrenamos la búsqueda sobre los datos de entrenamiento balanceados y reducidos
grid_search.fit(X_train_final, y_train_balanced)

print(f"Mejores Parámetros encontrados: {grid_search.best_params_}")
print(f"Mejor Score F1 (CV): {grid_search.best_score_:.4f}")

# 3e. Obtener el mejor modelo
best_model = grid_search.best_estimator_

# 3f. Evaluación e Interpretación del Modelo Final
print("\n--- Evaluación del Modelo FINAL Optimizado ---")
# Predecimos sobre el X_test_final (reducido, pero no balanceado)
y_pred_final = best_model.predict(X_test_final)

accuracy = accuracy_score(y_test, y_pred_final)
print(f"Precisión (Accuracy) - Modelo Final: {accuracy * 100:.2f}%")

print("\nReporte de Clasificación - Modelo Final:")
# Reportamos contra el y_test original
print(classification_report(y_test, y_pred_final))

# Imprimir la Matriz de Confusión en texto
cm = confusion_matrix(y_test, y_pred_final)
print("\nMatriz de Confusión - Modelo FINAL:")
print(cm)

Librerías importadas.

--- 1. Cargando y Explorando los Datos ---
Archivo 'data/water_potability.csv' cargado exitosamente.

--- Información General (Tipos de Dato y Conteo) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               2785 non-null   float64
 1   Hardness         3276 non-null   float64
 2   Solids           3276 non-null   float64
 3   Chloramines      3276 non-null   float64
 4   Sulfate          2495 non-null   float64
 5   Conductivity     3276 non-null   float64
 6   Organic_carbon   3276 non-null   float64
 7   Trihalomethanes  3114 non-null   float64
 8   Turbidity        3276 non-null   float64
 9   Potability       3276 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 256.1 KB

--- Conteo de Valores Nulos (Missing) ---
ph                 491
Hardness             0
Solids               0